In [55]:
import os
import pandas as pd
from sklearn import datasets, linear_model
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

import matplotlib.pyplot as plt

In [18]:
raw_df = pd.read_csv('integrated_data.csv', low_memory=False)
raw_df

,player_name,pitcher,game_date,game_pk,pitch_of_game,at_bat_number,pitch_number,pitch_type,pitch_name,release_speed,...,az,sz_top,sz_bot,effective_speed,release_spin_rate,release_extension,release_pos_y,spin_axis,game_precedes_injury,ID
0,"Scherzer, Max",453286,2020-07-23,630851,1,1,1,FF,4-Seam Fastball,95.3,...,-15.689358,3.85,1.84,95.7,2484.0,6.3,54.21,226.0,False,0
1,"Scherzer, Max",453286,2020-07-23,630851,2,1,2,FF,4-Seam Fastball,96.1,...,-12.476899,3.58,1.75,95.9,2477.0,6.1,54.38,226.0,False,0
2,"Scherzer, Max",453286,2020-07-23,630851,3,1,3,FF,4-Seam Fastball,96.7,...,-14.099707,3.33,1.65,96.4,2421.0,6.2,54.29,224.0,False,0
3,"Scherzer, Max",453286,2020-07-23,630851,4,1,4,CU,Curveball,79.7,...,-39.867212,3.51,1.65,79.6,2846.0,6.2,54.28,60.0,False,0
4,"Scherzer, Max",453286,2020-07-23,630851,5,1,5,FF,4-Seam Fastball,96.1,...,-14.962333,3.41,1.56,96.3,2450.0,6.4,54.09,227.0,False,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2686486,"Pérez, Cionel",672335,2024-05-02,747044,273,71,1,SI,Sinker,96.7,...,-18.790202,3.37,1.62,96.3,2142.0,6.0,54.52,144.0,False,80501
2686487,"Pérez, Cionel",672335,2024-05-02,747044,274,71,2,CU,Curveball,82.8,...,-41.939864,3.36,1.61,82.2,2669.0,6.0,54.53,330.0,False,80501
2686488,"Pérez, Cionel",672335,2024-05-02,747044,275,71,3,SI,Sinker,96.6,...,-23.722864,3.31,1.57,96.2,2111.0,6.0,54.53,146.0,False,80501
2686489,"Pérez, Cionel",672335,2024-05-02,747044,276,71,4,SL,Slider,84.6,...,-41.547310,3.30,1.57,84.0,2628.0,6.0,54.49,331.0,False,80501


In [19]:
speed_df = raw_df[['player_name','pitcher', 'game_date', 'game_pk', 'ID', 'pitch_of_game', 'release_speed', 'game_precedes_injury']]
speed_df

,player_name,pitcher,game_date,game_pk,ID,pitch_of_game,release_speed,game_precedes_injury
0,"Scherzer, Max",453286,2020-07-23,630851,0,1,95.3,False
1,"Scherzer, Max",453286,2020-07-23,630851,0,2,96.1,False
2,"Scherzer, Max",453286,2020-07-23,630851,0,3,96.7,False
3,"Scherzer, Max",453286,2020-07-23,630851,0,4,79.7,False
4,"Scherzer, Max",453286,2020-07-23,630851,0,5,96.1,False
...,...,...,...,...,...,...,...,...
2686486,"Pérez, Cionel",672335,2024-05-02,747044,80501,273,96.7,False
2686487,"Pérez, Cionel",672335,2024-05-02,747044,80501,274,82.8,False
2686488,"Pérez, Cionel",672335,2024-05-02,747044,80501,275,96.6,False
2686489,"Pérez, Cionel",672335,2024-05-02,747044,80501,276,84.6,False


In [63]:
speed_df[speed_df['player_name'] == 'Milner, Hoby']

,player_name,pitcher,game_date,game_pk,ID,pitch_of_game,release_speed,game_precedes_injury
2233,"Milner, Hoby",571948,2020-07-24,631182,67,342,77.2,False
23127,"Milner, Hoby",571948,2020-07-29,631389,753,186,87.7,False
23128,"Milner, Hoby",571948,2020-07-29,631389,753,187,78.7,False
23129,"Milner, Hoby",571948,2020-07-29,631389,753,188,87.5,False
23130,"Milner, Hoby",571948,2020-07-29,631389,753,189,88.5,False
...,...,...,...,...,...,...,...,...
2678086,"Milner, Hoby",571948,2024-04-30,745997,80241,254,78.9,False
2678087,"Milner, Hoby",571948,2024-04-30,745997,80241,255,78.3,False
2678088,"Milner, Hoby",571948,2024-04-30,745997,80241,256,87.4,False
2678089,"Milner, Hoby",571948,2024-04-30,745997,80241,257,78.2,False


In [22]:
speed_df[speed_df['game_precedes_injury'] == True]


,player_name,pitcher,game_date,game_pk,ID,pitch_of_game,release_speed,game_precedes_injury
1216,"Holmes, Clay",605280,2020-07-24,630989,31,165,79.2,True
1217,"Holmes, Clay",605280,2020-07-24,630989,31,166,80.2,True
1218,"Holmes, Clay",605280,2020-07-24,630989,31,167,80.0,True
1219,"Holmes, Clay",605280,2020-07-24,630989,31,168,80.3,True
1220,"Holmes, Clay",605280,2020-07-24,630989,31,169,93.9,True
...,...,...,...,...,...,...,...,...
2635930,"Puk, A.J.",640462,2024-04-19,746892,79049,103,78.7,True
2635931,"Puk, A.J.",640462,2024-04-19,746892,79049,104,91.3,True
2635932,"Puk, A.J.",640462,2024-04-19,746892,79049,105,85.7,True
2635933,"Puk, A.J.",640462,2024-04-19,746892,79049,106,90.5,True


In [34]:
speed_df['ID'].unique()

array([    0,     1,     2, ..., 80499, 80500, 80501])

In [23]:
first = speed_df[speed_df['ID'] == 0]
first



,player_name,pitcher,game_date,game_pk,ID,pitch_of_game,release_speed,game_precedes_injury
0,"Scherzer, Max",453286,2020-07-23,630851,0,1,95.3,False
1,"Scherzer, Max",453286,2020-07-23,630851,0,2,96.1,False
2,"Scherzer, Max",453286,2020-07-23,630851,0,3,96.7,False
3,"Scherzer, Max",453286,2020-07-23,630851,0,4,79.7,False
4,"Scherzer, Max",453286,2020-07-23,630851,0,5,96.1,False
...,...,...,...,...,...,...,...,...
169,"Scherzer, Max",453286,2020-07-23,630851,0,170,93.7,False
170,"Scherzer, Max",453286,2020-07-23,630851,0,171,94.8,False
171,"Scherzer, Max",453286,2020-07-23,630851,0,172,74.9,False
172,"Scherzer, Max",453286,2020-07-23,630851,0,173,85.1,False


In [25]:
predictor = first[['pitch_of_game']]
response = first[['release_speed']]

regr = linear_model.LinearRegression()
regr.fit(predictor, response)

print(regr.coef_)

[[-0.01492825]]


In [26]:
injured = speed_df[speed_df['ID'] == 31]
injured

,player_name,pitcher,game_date,game_pk,ID,pitch_of_game,release_speed,game_precedes_injury
1216,"Holmes, Clay",605280,2020-07-24,630989,31,165,79.2,True
1217,"Holmes, Clay",605280,2020-07-24,630989,31,166,80.2,True
1218,"Holmes, Clay",605280,2020-07-24,630989,31,167,80.0,True
1219,"Holmes, Clay",605280,2020-07-24,630989,31,168,80.3,True
1220,"Holmes, Clay",605280,2020-07-24,630989,31,169,93.9,True
1245,"Holmes, Clay",605280,2020-07-24,630989,31,194,91.0,True
1246,"Holmes, Clay",605280,2020-07-24,630989,31,195,78.3,True
1247,"Holmes, Clay",605280,2020-07-24,630989,31,196,91.4,True
1248,"Holmes, Clay",605280,2020-07-24,630989,31,197,91.7,True
1249,"Holmes, Clay",605280,2020-07-24,630989,31,198,79.3,True


In [30]:
predictor = injured[['pitch_of_game']]
response = injured[['release_speed']]

regr = linear_model.LinearRegression()
regr.fit(predictor, response)

regr.coef_[0][0]

0.021836199821745284

In [50]:
final_df = pd.DataFrame()
i = 0
for id in speed_df['ID'].unique():
    id_df = speed_df[speed_df['ID'] == id]
    
    predictor = id_df[['pitch_of_game']]
    response = id_df[['release_speed']]
    
    regr = linear_model.LinearRegression()
    regr.fit(predictor, response)
    
    slope = regr.coef_[0][0]
    mean = id_df[['release_speed']].mean(axis=1)
    std = id_df[['release_speed']].std()['release_speed']
    
    to_add_df = id_df[['player_name', 'pitcher', 'game_date', 'game_pk', 'ID', 'game_precedes_injury']].drop_duplicates()
    to_add_df['slope'] = slope
    to_add_df['mean'] = mean
    to_add_df['std'] = std
    
    final_df = pd.concat([final_df, to_add_df])
    
final_df

,player_name,pitcher,game_date,game_pk,ID,game_precedes_injury,slope,mean,std
0,"Scherzer, Max",453286,2020-07-23,630851,0,False,-0.014928,95.3,5.781597
22,"Cole, Gerrit",543037,2020-07-23,630851,1,False,-0.024393,96.3,4.915894
174,"May, Dustin",669160,2020-07-23,631377,2,False,-0.049270,98.7,4.116826
186,"Cueto, Johnny",456501,2020-07-23,631377,3,False,0.002558,90.7,4.889891
297,"Ferguson, Caleb",657571,2020-07-23,631377,4,False,-1.540000,94.3,3.850065
...,...,...,...,...,...,...,...,...,...
2686382,"Marinaccio, Ron",676760,2024-05-02,747044,80497,False,-0.021832,92.6,5.905216
2686420,"Coulombe, Danny",543056,2024-05-02,747044,80498,False,-0.063261,85.2,3.924321
2686438,"Webb, Jacob",657097,2024-05-02,747044,80499,False,-0.027017,81.2,5.235542
2686441,"Tonkin, Michael",543859,2024-05-02,747044,80500,False,-0.065535,93.8,5.785890


In [54]:
final_df2 = final_df[['slope', 'mean', 'std', 'game_precedes_injury']]
final_df2

,slope,mean,std,game_precedes_injury
0,-0.014928,95.3,5.781597,False
22,-0.024393,96.3,4.915894,False
174,-0.049270,98.7,4.116826,False
186,0.002558,90.7,4.889891,False
297,-1.540000,94.3,3.850065,False
...,...,...,...,...
2686382,-0.021832,92.6,5.905216,False
2686420,-0.063261,85.2,3.924321,False
2686438,-0.027017,81.2,5.235542,False
2686441,-0.065535,93.8,5.785890,False


In [61]:
final_df[final_df.isna().any(axis=1)]

,player_name,pitcher,game_date,game_pk,ID,game_precedes_injury,slope,mean,std
2233,"Milner, Hoby",571948,2020-07-24,631182,67,False,0.0,77.2,NaN
4055,"Wahl, Bobby",592833,2020-07-24,631641,125,False,0.0,92.3,NaN
16417,"Harper, Ryne",607560,2020-07-28,630866,534,False,0.0,74.2,NaN
25041,"Greene, Shane",572888,2020-07-30,630346,815,False,0.0,79.2,NaN
35578,"McFarland, T.J.",519008,2020-08-02,631068,1140,False,0.0,86.6,NaN
...,...,...,...,...,...,...,...,...,...
2589785,"Antone, Tejay",622088,2024-04-07,746734,77732,True,0.0,89.0,NaN
2609662,"Contreras, Roansy",672710,2024-04-13,745594,78302,False,0.0,94.7,NaN
2636683,"Martinez, Seth",661527,2024-04-20,744865,79072,False,0.0,89.6,NaN
2638228,"Alvarado, José",621237,2024-04-20,745590,79116,False,0.0,96.5,NaN


In [57]:


predictor = final_df2[['slope', 'mean', 'std']]
response = final_df2[['game_precedes_injury']]

X_train, X_test, y_train, y_test = train_test_split(predictor, response, test_size=0.20, random_state=42)

log_regr = linear_model.LogisticRegression()

log_regr.fit(X_train, y_train)

y_pred = log_regr.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f'MSE: {mse} | R2: {r2}')



ValueError: Input X contains NaN.
LogisticRegression does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values